# EDA

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

DEBUG = False
SEED = 666

## Load and format all the datasets

### Accounts dataset

#### Import test and train datasets

In [2]:
df_accounts_train = pd.read_csv(f"orig/accounts_train.csv")

In [3]:
df_accounts_test = pd.read_csv(f"orig/accounts_test.csv")

#### Function for formatting account datasets 

In [4]:
def format_accounts(df_accounts):
    print("Number of different Accounts : {} ".format(df_accounts.Account.nunique()))
    df_accounts = df_accounts.pivot_table(index='Customer', columns='Account', aggfunc='size', fill_value=0)
    print(df_accounts.shape)
    print("Number of NANs : {}".format(df_accounts.isna().sum().sum()))

    return df_accounts

#### Formatting train dataset

In [5]:
df_accounts_train = format_accounts(df_accounts_train)
df_accounts_train.head(1)

Number of different Accounts : 8 
(6577, 8)
Number of NANs : 0


Account,Credit Card,Current,Deposit,Investment,Joint,Loan,Mortgage,On Demand Deposit
Customer,,,,,,,,
708115009,1,0,0,0,0,0,0,0


#### Formatting test dataset

In [6]:
df_accounts_test = format_accounts(df_accounts_test)
df_accounts_test.head(1)

Number of different Accounts : 8 
(3542, 8)
Number of NANs : 0


Account,Credit Card,Current,Deposit,Investment,Joint,Loan,Mortgage,On Demand Deposit
Customer,,,,,,,,
708182832,1,1,1,1,0,0,0,0


### Customers Dataset

#### Import test and train datasets

In [7]:
df_customers_train = pd.read_csv(f"orig/customers_train.csv")

In [8]:
df_customers_test = pd.read_csv(f"orig/customers_test.csv")

#### Function for formatting account dataset 

In [9]:
def format_customers(df_customers):
    df_customers[['Start_Year', 'Start_Day', 'Start_Month']] = df_customers['Start_Date'].str.split('-', expand=True)

    print("Unique Start Months : {} ".format(df_customers.Start_Month.nunique()))

    df_customers.drop(['Start_Date','Start_Month'], axis=1, inplace=True)

    df_customers = df_customers.set_index('Customer')

    print(df_customers.shape)
    print("Number of NANs : {}".format(df_customers.isna().sum().sum()))
    
    return df_customers

#### Formatting train dataset

In [10]:
df_customers_train = format_customers(df_customers_train)
df_customers_train.head(1)
#df_customers_train.Marital_Status.nunique()

Unique Start Months : 1 
(6577, 15)
Number of NANs : 761


,Churn,Card,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income,Start_Year,Start_Day
Customer,,,,,,,,,,,,,,,
797197508,No,Silver,1,2315.0,1565,750.0,0.676,48.0,F,Graduate,Married,2,Less than €30K,2020,06


#### Formatting test dataset

In [11]:
df_customers_test = format_customers(df_customers_test)
df_customers_test.head(1)
#df_customers_test.Marital_Status.nunique()

Unique Start Months : 1 
(3542, 14)
Number of NANs : 0


,Card,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income,Start_Year,Start_Day
Customer,,,,,,,,,,,,,,
774663629,Gold,2,29663.0,1743,27920.0,0.059,51,M,Graduate,Single,3,€50K - €80K,2020,01


#### Comment
- Test dataset has one more marital status - Unknown
- We will add this to the marital status category after we transform all the object types to categories

#### Cleaning NANs in training data

In [12]:
df_customers_train.dropna(inplace=True)
print("Number of NANs : {}".format(df_customers_train.isna().sum().sum()))
print(df_customers_train.shape)
df_customers_train.head(5)

Number of NANs : 0
(5831, 15)


,Churn,Card,Customer_Service_Calls,Credit_Limit,Total_Revolving_Balance,Average_Open_To_Buy,Average_Utilisation_Ratio,Age,Gender,Education,Marital_Status,Dependents,Income,Start_Year,Start_Day
Customer,,,,,,,,,,,,,,,
797197508,No,Silver,1,2315.0,1565,750.0,0.676,48.0,F,Graduate,Married,2,Less than €30K,2020,06
812854728,No,Silver,1,7645.0,2076,5569.0,0.272,58.0,F,Second level,Single,5,Less than €30K,2020,01
768000743,No,Silver,2,6394.0,0,6394.0,0.000,55.0,F,Second level,Single,2,Less than €30K,2020,01
759029725,No,Silver,4,1879.0,1486,393.0,0.791,37.0,F,Second level,Single,1,Less than €30K,2021,02
764982590,No,Silver,4,6036.0,1409,4627.0,0.233,56.0,M,PhD,Married,2,€50K - €80K,2018,10


### Transactions Dataset

#### Import test and train datasets

In [13]:
df_transaction_train = pd.read_csv(f"orig/transactions_train.csv")

In [14]:
df_transaction_test = pd.read_csv(f"orig/transactions_test.csv")

#### Function for formatting transaction dataset 

In [15]:
def format_transactions(df_transaction):
    df_transaction['Date'] = pd.to_datetime(df_transaction['Date'])

    df_transaction['Month'] = df_transaction['Date'].dt.month

    df_transaction_grouped = df_transaction.groupby(['Customer', 'Month'])['Amount'].sum().reset_index()

    df_transaction_pivot = df_transaction_grouped.pivot(index='Customer', columns='Month', values='Amount').reset_index()

    df_transaction_pivot.columns = ['Customer', 'Jan_Total', 'Feb_Total', 'Mar_Total', 'Apr_Total', 'May_Total', 'Jun_Total', 'Jul_Total', 'Aug_Total', 'Sep_Total', 'Oct_Total', 'Nov_Total', 'Dec_Total']

    df_transaction = df_transaction_pivot.set_index('Customer')

    df_transaction = df_transaction.fillna(0)
    
    return df_transaction

#### Formatting train dataset

In [16]:
df_transaction_train = format_transactions(df_transaction_train)
df_transaction_train.head(1)

,Jan_Total,Feb_Total,Mar_Total,Apr_Total,May_Total,Jun_Total,Jul_Total,Aug_Total,Sep_Total,Oct_Total,Nov_Total,Dec_Total
Customer,,,,,,,,,,,,
708115009,0.0,653.68,0.0,25.44,38.82,391.74,89.15,0.0,282.99,7.23,397.94,0.0


#### Formatting test dataset

In [17]:
df_transaction_test = format_transactions(df_transaction_test)
df_transaction_test.head(1)

,Jan_Total,Feb_Total,Mar_Total,Apr_Total,May_Total,Jun_Total,Jul_Total,Aug_Total,Sep_Total,Oct_Total,Nov_Total,Dec_Total
Customer,,,,,,,,,,,,
708182832,673.59,0.0,925.22,260.99,443.52,361.47,334.93,0.0,519.59,0.0,364.3,457.4


## Merge Datasets

### Merge train

In [18]:
df_train = pd.merge(df_accounts_train, df_customers_train, left_index=True, right_index=True)
df_train = pd.merge(df_train, df_transaction_train, left_index=True, right_index=True)
print(df_train.shape)
df_train.head(3)

(5831, 35)


,Credit Card,Current,Deposit,Investment,Joint,Loan,Mortgage,On Demand Deposit,Churn,Card,...,Mar_Total,Apr_Total,May_Total,Jun_Total,Jul_Total,Aug_Total,Sep_Total,Oct_Total,Nov_Total,Dec_Total
Customer,,,,,,,,,,,,,,,,,,,,,
708115009,1,0,0,0,0,0,0,0,Yes,Silver,...,0.00,25.44,38.82,391.74,89.15,0.00,282.99,7.23,397.94,0.00
708117651,1,1,0,0,0,0,0,0,No,Silver,...,1008.48,1114.80,412.19,513.07,818.03,1060.71,0.00,507.95,669.83,346.73
708163600,1,1,0,0,0,0,0,1,No,Silver,...,0.00,0.00,3794.27,0.00,561.84,1774.10,1081.42,1168.54,1077.05,325.53


### Merge test

In [19]:
df_test = pd.merge(df_accounts_test, df_customers_test, left_index=True, right_index=True)
df_test = pd.merge(df_test, df_transaction_test, left_index=True, right_index=True)
print(df_test.shape)
df_test.head(3)

(3542, 34)


,Credit Card,Current,Deposit,Investment,Joint,Loan,Mortgage,On Demand Deposit,Card,Customer_Service_Calls,...,Mar_Total,Apr_Total,May_Total,Jun_Total,Jul_Total,Aug_Total,Sep_Total,Oct_Total,Nov_Total,Dec_Total
Customer,,,,,,,,,,,,,,,,,,,,,
708182832,1,1,1,1,0,0,0,0,Silver,1,...,925.22,260.99,443.52,361.47,334.93,0.00,519.59,0.00,364.30,457.40
708240660,1,1,1,1,0,1,0,0,Silver,3,...,111.69,0.00,0.00,456.52,285.88,17.58,81.31,24.92,521.61,15.67
708249778,1,1,0,1,0,0,1,0,Silver,2,...,306.64,0.00,709.69,0.00,0.00,606.68,0.00,191.04,0.00,118.69


## Clean

### Summarise dataset

In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5831 entries, 708115009 to 828317722
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Credit Card                5831 non-null   int64  
 1   Current                    5831 non-null   int64  
 2   Deposit                    5831 non-null   int64  
 3   Investment                 5831 non-null   int64  
 4   Joint                      5831 non-null   int64  
 5   Loan                       5831 non-null   int64  
 6   Mortgage                   5831 non-null   int64  
 7   On Demand Deposit          5831 non-null   int64  
 8   Churn                      5831 non-null   object 
 9   Card                       5831 non-null   object 
 10  Customer_Service_Calls     5831 non-null   int64  
 11  Credit_Limit               5831 non-null   float64
 12  Total_Revolving_Balance    5831 non-null   int64  
 13  Average_Open_To_Buy        5831 non

In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3542 entries, 708182832 to 828331426
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Credit Card                3542 non-null   int64  
 1   Current                    3542 non-null   int64  
 2   Deposit                    3542 non-null   int64  
 3   Investment                 3542 non-null   int64  
 4   Joint                      3542 non-null   int64  
 5   Loan                       3542 non-null   int64  
 6   Mortgage                   3542 non-null   int64  
 7   On Demand Deposit          3542 non-null   int64  
 8   Card                       3542 non-null   object 
 9   Customer_Service_Calls     3542 non-null   int64  
 10  Credit_Limit               3542 non-null   float64
 11  Total_Revolving_Balance    3542 non-null   int64  
 12  Average_Open_To_Buy        3542 non-null   float64
 13  Average_Utilisation_Ratio  3542 non

#### Comments
Issues:
 - Account,Card,Gender,Education,Marital_Status,Income can be encoded to catgerical values
 - Churn (target) can be encoded to a boolean

### Encoding Object Values

#### All features with object data types

In [22]:
for c in df_train.columns:
    if df_train[c].dtype == "object":
        df_train[c] = pd.Categorical(df_train[c])

In [23]:
for c in df_test.columns:
    if df_test[c].dtype == "object":
        df_test[c] = pd.Categorical(df_test[c])

In [24]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5831 entries, 708115009 to 828317722
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Credit Card                5831 non-null   int64   
 1   Current                    5831 non-null   int64   
 2   Deposit                    5831 non-null   int64   
 3   Investment                 5831 non-null   int64   
 4   Joint                      5831 non-null   int64   
 5   Loan                       5831 non-null   int64   
 6   Mortgage                   5831 non-null   int64   
 7   On Demand Deposit          5831 non-null   int64   
 8   Churn                      5831 non-null   category
 9   Card                       5831 non-null   category
 10  Customer_Service_Calls     5831 non-null   int64   
 11  Credit_Limit               5831 non-null   float64 
 12  Total_Revolving_Balance    5831 non-null   int64   
 13  Average_Open_To_Buy 

In [25]:
df_train.Marital_Status = df_train.Marital_Status.cat.add_categories(['Unknown'])
print(df_train.Marital_Status.cat.categories)

Index(['Divorced', 'Married', 'Single', 'Unknown'], dtype='object')


### Check nans

In [26]:
df_train.isna().sum().sum()

0

In [27]:
df_test.isna().sum().sum()

0

## Save test and train datasets

In [28]:
df_train.to_pickle("./orig/data/churn_train.pkl")

In [29]:
df_test.to_pickle("./orig/data/churn_test.pkl")